# Hyper-parameter tuning in model training
Ngoc-Bien NGUYEN et Xian YANG, février 2018   
Répertoire Github : https://github.com/MarchesLearning/salaryDiscovery

## 0. Induction du rapport
Dans les TP de Machine Learning on a vu une série d'algorithmes de base pour effectuer des tâches supervisées. Or là, on a fait de façon à donner des valeurs arbitraires aux hyper-paramètres du modèle qu'on considère raisonnables juste pour constater le fonctionnement des algorithmes. Le tunnage des hyper-paramètres est considéré aujourd'hui comme un art. Dans ce projet de machine learning, au contraire de celui de data mining, on ne découvre pas de nouveaux algorithmes mais on se concentre plutôt sur un aspect technique mais crucial de machine learning, le choix de valeurs des hyper-paramètres.  
  
  On continue à travailler avec notre jeu de données utilisé auparavant, appelé "adult dataset", dont la description officielle se trouve [ici](http://www.cs.utoronto.ca/~delve/data/adult/desc.html), issu d'un certain bureau de recensement gouvernemental américain. Ce jeu de données a relevé, pour 48.842 sujets de l'enquête apparamment tous résidant aux Etats-Unis, leur situation familiale, parcours académique, profession, âge, race, sex, pays d'origine, temps de travail hebdomadaire, fortune personnelle et, ce qui est le plus important ici, leur salaire. Ce dernier n'est néanmoins pas une variable continue. Elle est par contre binaire, c'est-à-dire que les sujets selon leur salaire annuel dépassant ou pas 50K sont classés dans deux sous-groupes. Celle-ci étant la variable à prédire dans notre tâche, les autres variables sont toutes considérées indépendantes et sont mélangées quantitatives et qualitatives. 
  
  Dans le projet précédent on a discuté prudemment le pré-traitement des données brutes y compris la _feature selection_, le traitement des données manquantes et la transformation one-hot des variables catégoriques. On avait effectué cette procédure toujours en faveur du taux de bien classés (accuracy) de la prédiction dont les détails ne seront pas répétés dans cet article. En résumé, les données qu'on va utiliser ici sont toutes numériques, les modalités des variables catégoriques déjà transformées de la manière one-hot.   
  
  L'article est organisé de façon suivante : 

## 1. Mode d'entraînement et d'évaluation
Comme d'habitude, on découpe le jeu de données entier en jeu d'entraînement (training set) et jeu de test (test set) à un taux 8:2 de façon aléatoire. Puis dans chaque méthode supervisée utilisée, on va faire une validation croisée à priori à 5 plis (5-folds) sur le jeu d'entraînement pour déterminer le meilleur n-uplet d'hyper-paramètres ayant le taux de bien classés le plus élevé. Ensuite on applique cette méthode sur le jeu de test jamais vu dans l'entraînement pour en obtenir un score final.  
  
  La fonction de `sklearn` qu'on a utilisé pour le découpage, c'est `train_test_split` du module `sklearn.model_selection`. Elle s'occupe tout seule du tirage aléatoire.

## 2. Initiation de la question : recherche exhaustive des hyper-paramètres optimisés et l'algorithme KNN
On commence par regarder le modèle le plus intuitif en machine learning, celui de KNN. Dans un context normal, il y a un seul hyper-paramètre à déterminer dans cette méthode : la valeur de k. Bien que dans des méthodes avancées certains prennent encore en considération la distribution du poid pour les voisins, l'algorithme implémenté etc., ceux-ci ne sont pas au premier rang de notre étude.   
  
  Comment trouver cette valeur de k optimisée ? La façon banale est bien sûr de faire à la main (des boucles p. ex.) en parcourant toutes les valeurs impaires possibles de k jusqu'à une vingtaine (puisqu'on est dans une question de classification binaire). Toutefois, `sklearn` nous a proposé une classe `GridsearchCV` pour définir la grille à parcourir (icic tout simplement les valeurs de k), la forme de validation croisée à effectuer qui fera de façon automatique la recherche de valeurs d'hyper-paramètres optimisées. 
  
  La valeur optimisée de k qu'on a trouvée sur le jeu d'entrainement est finalement 9.

## 3. Recherche par grille : une implémentation particulière pour la régression logistique
La regression logistique fait partie des méthodes les plus fondamentales dans la question de classification binaire et elle est tant robuste que pertinente pour notre jeu de données, puisque le taux de nombre d'observations par rapport au nombre de variables est plus grand que 1.000. Dans ce cas-là, la régression logistique est une des méthodes qu'il faut essayer en premier temps pour former un certain genre de benchmark. Trois hyper-paramètres principaux sont à tuner dans cette méthode : la pénalté à utiliser (L-1 ou L-2), son amplitude C, l'algo de recherche de la solution. Comme avant avec cet espace d'hyper-paramètres pas très énorme (puisque deux 'entre trois sont juste une valeur catégorique voire binaire), on peut toujours employer la classe `GridSearchCV`. Néanmoins, `sklearn` a proposé une classe spéciale `LogisticRegressionCV` qui implémente exactement la même stratégie de recherche mais exclusivement pour la regression logistique. L'utilité de cette classe c'est une grande économie en termes de temps (ça a fait presque 4 fois plus rapide dans notre entraînement). N'ayant pas eu le temps d'étudier la raison dernière, on a remarqué toutefois que si on met bien un domaine d'hyper-paramètres à rechercher identique, les deux approches doivent toujours donner des résultats identiques, à une différence autorisée à pré-définir (1e-4 p. ex.) près.

## 4. Optimisation randomisée des hyper-paramètres et la forêt aléatoire
Le tunnage des hyper-paramètres commence à devenir compliqué lorsqu'il vient aux forêts aléatoires. On doit déterminer entre autres :   
  * le nombre maximum de variables à considérer à chaque itération, où les choix habituels sont $\sqrt n$, $\log_2 n$ et $1$ (le cas de la forêt extrêmement aléatoire),
  * le profondeur miximum d'un arbre,
  * la taille mminimum de la feuille,
  * l'emploi de Bootstrap ou non,
  * l'emploi du critère dans la fonction de perte, Gini ou entropy et
  * le nombre d'arbres dans la forêt.
  
Avec un tel espace d'hyper-paramètres un peu grand, on voit que l'entraînement par la méthode de recherche par grille prend énormément de temps (par rapport au type et à la taille du jeu de données). Que peut-on faire de mieux dans ce cas-là ? `sk-learn` nous propose également une classe `RandomisedSearchCV` qui implémente une stratégie de recherche randomisée. Non seulement peut cet algorithme de recherche gagner une grande partie de temps, mais il peut aussi 